In [2]:
import torch
import torch.nn as nn
import numpy as np
from scipy.io import loadmat
from tqdm import tqdm
from pyDOE import lhs
import torch.optim as optim

In [3]:
class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                if i % 2 == 0
                else nn.Tanh()
                for i in range(10)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        
        # Trainable parameter for the wave number squared (k^2)
        self.k2 = nn.Parameter(torch.tensor([30.0], dtype=torch.float32, device="cuda"))

        # Optimizer
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        self.optimizer.param_groups[0]["params"].append(self.k2)
        
        self.loss = nn.MSELoss()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    # def residual_loss(self, xtrain):
    #     g = xtrain.clone()
    #     g.requires_grad = True
    #     u_pred = self.forward(g)
        
    #     # Compute gradients
    #     u_grad = torch.autograd.grad(
    #         u_pred, g, torch.ones_like(u_pred), retain_graph=True, create_graph=True
    #     )[0]
    #     u_lap = torch.autograd.grad(
    #         u_grad, g, torch.ones_like(u_grad), create_graph=True
    #     )[0].sum(dim=1, keepdim=True)
        
    #     # Residual form of the Helmholtz equation
    #     residual = u_lap + self.k2 * u_pred
    #     return self.loss(residual, torch.zeros_like(residual))
    #     THIS IS VERY STUPID OMFG LET'S TRY SOMETHING ELSE CUZ THE GODS HATE ME AND LIFE IS AN ENDLESS PIT OF DESPAIR THAT IS VOID OF ALL HAPPINESS
    
    def residual_loss(self, xtrain, fhat):
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        
        # Compute gradients
        u_grad = torch.autograd.grad(
            u_pred, g, torch.ones_like(u_pred), create_graph=True, retain_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_grad[:, [0]], g, torch.ones_like(u_grad[:, [0]]), create_graph=True
        )[0][:, [0]]
        u_tt = torch.autograd.grad(
            u_grad[:, [1]], g, torch.ones_like(u_grad[:, [1]]), create_graph=True
        )[0][:, [1]]
        
        # Residual calculation
        residual = u_xx + u_tt + self.k2 * u_pred - fhat
        return torch.mean(residual**2)  # Mean squared error for the residual


    def total_loss(self, xtrain, utrain):
        alpha_female = 10.0
        data_loss = self.loss_fn(xtrain, utrain)  # Match observed data
        physics_loss = self.residual_loss(xtrain)  # Enforce governing equations
        return data_loss + alpha_female* physics_loss

    def train_model(self, xtrain, utrain, epochs=10000):
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain)
            loss.backward()
            self.optimizer.step()
            
            # Logging
            if epoch % 1000 == 0:
                print(
                    f"Epoch {epoch}, Loss {loss.item()}, "
                    f"k^2 (Wave Number Squared) {self.k2.item()}"
                )

In [12]:
# again.

class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                if i % 2 == 0
                else nn.Tanh()
                for i in range(10)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        
        # Trainable parameter for the wave number squared (k^2)
        self.k2 = nn.Parameter(torch.tensor([1.0], dtype=torch.float32)) 

        

        # Optimizer
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)  # Larger learning rate
        # self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=1000, gamma=0.9)  # Scheduler to reduce LR

        self.loss = nn.MSELoss()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        # Physics-informed loss based on the Helmholtz equation
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        
        # Compute the gradients for second derivatives (u_xx and u_tt)
        u_grad = torch.autograd.grad(
            u_pred, g, torch.ones_like(u_pred), create_graph=True, retain_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_grad[:, [0]], g, torch.ones_like(u_grad[:, [0]]), create_graph=True
        )[0][:, [0]]
        u_tt = torch.autograd.grad(
            u_grad[:, [1]], g, torch.ones_like(u_grad[:, [1]]), create_graph=True
        )[0][:, [1]]
        
        # Residual calculation for Helmholtz equation: u_xx + u_tt + k^2 * u = fhat
        residual = u_xx + u_tt + self.k2 * u_pred - fhat
        return self.loss(residual, fhat)

    def total_loss(self, xtrain, utrain):
        fhat = torch.zeros(xtrain.shape[0], 1)
        alpha_female = 10.0
        data_loss = self.loss_fn(xtrain, utrain)  # Match observed data
        physics_loss = self.residual_loss(xtrain, fhat)  # Enforce governing equations
        return data_loss + alpha_female * physics_loss

    def train_model(self, xtrain, utrain, epochs=10000):
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain)
            loss.backward()
            self.optimizer.step()
            # self.scheduler.step()
            
            # Logging
            if epoch % 1000 == 0:
                print(
                    f"Epoch {epoch}, Loss {loss.item()}, "
                    f"k^2 (Wave Number Squared) {self.k2.item()}"
                )

In [6]:
u = np.load("C:/Users/ASUS/NoisyICML/pinns-inverse/Helmholtz/helmholtz_solution.npy")  # BOB THE BUILDER
x = np.load("C:/Users/ASUS/NoisyICML/pinns-inverse/Helmholtz/x_coordinate.npy")  # BOB THE BUILDER
t = np.load("C:/Users/ASUS/NoisyICML/pinns-inverse/Helmholtz/t_coordinate.npy")[:-1]

In [7]:
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.float32)
u = torch.tensor(u, dtype=torch.float32)

In [8]:
X, T = np.meshgrid(x, t)
xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
utrue = u.flatten()[:, None]

In [10]:
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available

2.2.1+cpu
False


In [13]:
idx = np.random.choice(xtrue.shape[0], 10000, replace=False)
xtrain = xtrue[idx, :]
utrain = utrue[idx, :]

Xtrain = torch.tensor(xtrain, dtype=torch.float32)
Utrain = torch.tensor(utrain, dtype=torch.float32)

model = PINN(input_size=2, hidden_size=20, output_size=1)
model.train_model(Xtrain, Utrain, epochs=20000)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_19528\369146865.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Utrain = torch.tensor(utrain, dtype=torch.float32)
  0%|          | 3/20000 [00:00<22:16, 14.96it/s]

Epoch 0, Loss 0.6556071043014526, k^2 (Wave Number Squared) 0.9900000095367432


  3%|▎         | 666/20000 [00:48<23:18, 13.83it/s]  


KeyboardInterrupt: 

In [9]:
model.train_model(Xtrain, Utrain, epochs=10000)

  0%|          | 39/10000 [00:00<00:49, 199.82it/s]

Epoch 0, Loss 0.24674396216869354, k^2 (Wave Number Squared) -3.5944936275482178


 10%|█         | 1037/10000 [00:04<00:38, 231.60it/s]

Epoch 1000, Loss 0.24692048132419586, k^2 (Wave Number Squared) -3.796750545501709


 20%|██        | 2028/10000 [00:09<00:37, 211.38it/s]

...

 80%|████████  | 8025/10000 [00:40<00:10, 196.26it/s]

Epoch 8000, Loss 0.24660082161426544, k^2 (Wave Number Squared) -4.954023361206055


 90%|█████████ | 9026/10000 [00:45<00:05, 192.75it/s]

Epoch 9000, Loss 0.2465580701828003, k^2 (Wave Number Squared) -5.104205131530762


100%|██████████| 10000/10000 [00:50<00:00, 197.15it/s]


In [10]:
model.train_model(Xtrain, Utrain, epochs=20000)

  0%|          | 44/20000 [00:00<01:30, 220.93it/s]

Epoch 0, Loss 0.24655897915363312, k^2 (Wave Number Squared) -5.256725311279297


  5%|▌         | 1034/20000 [00:04<01:43, 183.07it/s]

Epoch 1000, Loss 0.24651500582695007, k^2 (Wave Number Squared) -5.409544944763184


 10%|█         | 2033/20000 [00:10<01:32, 194.71it/s]

...

 90%|█████████ | 18035/20000 [01:36<00:09, 196.70it/s]

Epoch 18000, Loss 0.24625959992408752, k^2 (Wave Number Squared) -7.487943649291992


 95%|█████████▌| 19041/20000 [01:40<00:04, 224.56it/s]

Epoch 19000, Loss 0.2462511956691742, k^2 (Wave Number Squared) -7.54771089553833


100%|██████████| 20000/20000 [01:45<00:00, 188.79it/s]
